# Composing Images for full Region Coverage

**Problem:** Some geografical regions need multiple swaths (satellite rows) to cover a geographical region, such that `ee.ImageCollection.first` in combination with `ee.Image.clip` leads to invalid images.

To overcome this problem, multiple images can be composed using `ee.ImageCollection.mosaic` or `ee.ImageCollection.median`.

# Setup
This problem appears when trying to retrieve surrounding satellite images for the training sample `airport 18` of fMoW, an airport in Lybia.

In [9]:
import ee
import geemap

EE_PROJECT_NAME = 'seeing-the-big-picture'

try:
    ee.Authenticate()
    ee.Initialize(project=EE_PROJECT_NAME)
except Exception as e:
    print("Please authenticate Earth Engine: earthengine authenticate")
    raise

# Rectange surrounding the training sample 'airport 18' from fMoW
bbox = [14.583450674625576, 27.177539780430607,
        14.718289694449945, 27.29808510902606]

center_coords = [27.237, 14.650]

region = ee.Geometry.Rectangle(bbox)

col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
       .filterBounds(region))

def scale_l8(image):
    return (image
            .select(['SR_B2', 'SR_B3', 'SR_B4'])
            .multiply(0.0000275)
            .add(-0.2))

scol = col.map(scale_l8)

# Problem
When selecting the least cloudy image of the collection, it might not fully cover the region of interest (ROI).

In [10]:
# Least cloudy image
least_cloudy = scol.sort('CLOUD_COVER').first().clip(region)

m = geemap.Map(center=center_coords, zoom=13)
m.addLayer(
    least_cloudy,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'Least Cloudy'
)
m.addLayerControl()
m

Map(center=[27.237, 14.65], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

# Compsing
The image collection contains all swaths that intersect with the ROI and can be composed into a single image. This can be helpful to avoid invalid images, but leads to longer compute time as potentially many images need to be considered.

## Median
Recuding the collection by taking the pixelwise median to cover the ROI makes it unlikely to include clouds and shadows as they have either very high or low pixel values. On the other hand, median compositions skew reality heavily as median pixel values are combined without further depedencies.

In [11]:
comp_median = scol.median().clip(region)

m = geemap.Map(center=center_coords, zoom=13)
m.addLayer(
    comp_median,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'Median'
)
m.addLayerControl()
m

Map(center=[27.237, 14.65], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

## Mosaic
Patching images of the collection using `ee.ImageCollection.mosaic` solves this problem, as spatial gaps are filled by cohesive swaths.

In [12]:
comp_mosaic = scol.mosaic().clip(region)

m = geemap.Map(center=center_coords, zoom=13)
m.addLayer(
    comp_mosaic,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'Mosaic'
)
m.addLayerControl()
m

Map(center=[27.237, 14.65], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

# Combination
As compositions take more compute resources, the least cloudy image can be assest by its fraction of valid pixels. If it contains more than $1 \%$ of invalid pixels, a mosaic composition can be computed.

In [13]:
least_cloudy = scol.sort('CLOUD_COVER').first()

rgb_mask = (least_cloudy
            .select(['SR_B2', 'SR_B3', 'SR_B4'])
            .mask()
            .reduce(ee.Reducer.min()))

coverage_dict = rgb_mask.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region,
    scale=30,
    maxPixels=1e7
)

least_cloudy_coverage = ee.Number(coverage_dict.get('min'))
rgb_ok = least_cloudy_coverage.gte(0.99)

composite = scol.mosaic()
context = ee.Image(ee.Algorithms.If(rgb_ok, least_cloudy, composite))

m = geemap.Map(center=center_coords, zoom=13)
m.addLayer(
    context,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'Combined Approach'
)
m.addLayerControl()
m

Map(center=[27.237, 14.65], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

# Swath intersections
`ee.ImageCollection.mosaic` might produce chiseled transitions at the intersection of satellite swaths. Those intersections might fall into ROIs in the worst case, making `ee.ImageCollection.mosaic` as the best choice questionable.

In [14]:
comp_median = scol.median()
comp_mosaic = scol.mosaic()

m = geemap.Map(center=[27.28718614141461, 15.179246277997064], zoom=12)

m.addLayer(
    comp_mosaic,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'Mosaic'
)

m.addLayer(
    comp_median,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'Median'
)
m.addLayerControl()
m

Map(center=[27.28718614141461, 15.179246277997064], controls=(WidgetControl(options=['position', 'transparent_…

# Remainder
However, the problem of cloudyness is not solved yet.

In [17]:
# Rectange surrounding the training sample 'airport 32' from fMoW
bbox = [-1.0140575646754744, -79.48694022837181, 
        -0.9691395896577617, -79.4420222533541]

center_coords = [-0.99, -79.46]
region = ee.Geometry.Rectangle(bbox)

col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
       .filterBounds(region))

def scale_l8(image):
    return (image
            .select(['SR_B2', 'SR_B3', 'SR_B4'])
            .multiply(0.0000275)
            .add(-0.2))

scol = col.map(scale_l8)

least_cloudy = scol.sort('CLOUD_COVER').first()

rgb_mask = (least_cloudy
            .select(['SR_B2', 'SR_B3', 'SR_B4'])
            .mask()
            .reduce(ee.Reducer.min()))

coverage_dict = rgb_mask.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region,
    scale=30,
    maxPixels=1e7
)

least_cloudy_coverage = ee.Number(coverage_dict.get('min'))
rgb_ok = least_cloudy_coverage.gte(0.99)

composite = scol.mosaic()
context = ee.Image(ee.Algorithms.If(rgb_ok, least_cloudy, composite))

m = geemap.Map(center=center_coords, zoom=13)
m.addLayer(
    context,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'Combined Approach'
)
m.addLayerControl()
m

EEException: Image.select: Parameter 'input' is required and may not be null.